## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-05-14-52-50 +0000,nyt,How the U.S. Focus on Fentanyl Helped Fuel a ‘...,https://www.nytimes.com/2025/12/05/world/ameri...
1,2025-12-05-14-51-31 +0000,nyt,Putin Basks in Praise From Modi on India Visit,https://www.nytimes.com/2025/12/05/world/asia/...
2,2025-12-05-14-50-28 +0000,nypost,State Street still using DEI to curry favor wi...,https://nypost.com/2025/12/05/business/state-s...
3,2025-12-05-14-50-21 +0000,nyt,But Who Is the Snake in ‘Zootopia 2’?,https://www.nytimes.com/2025/12/04/business/me...
4,2025-12-05-14-49-26 +0000,nypost,Horrific videos show teen driver speeding to 1...,https://nypost.com/2025/12/05/us-news/horrific...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2506/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
53,trump,51
201,new,24
6,putin,13
86,deal,12
233,he,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
220,2025-12-04-23-23-00 +0000,wsj,The Trump administration intends to formally s...,https://www.wsj.com/politics/policy/white-hous...,119
67,2025-12-05-11-42-00 +0000,wsj,The Supreme Court ruled that Texas could legal...,https://www.wsj.com/politics/policy/a-huge-red...,107
303,2025-12-04-19-19-46 +0000,nypost,Accused DC pipe-bomber Brian Cole Jr.’s nabbed...,https://nypost.com/2025/12/04/us-news/accused-...,104
278,2025-12-04-21-11-38 +0000,nypost,Trump boasts heads of Rwanda and Congo will be...,https://nypost.com/2025/12/04/us-news/trump-bo...,93
50,2025-12-05-12-19-01 +0000,cbc,U.S. Supreme Court paves way for midterm redis...,https://www.cbc.ca/news/world/us-scotus-texas-...,92


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
220,119,2025-12-04-23-23-00 +0000,wsj,The Trump administration intends to formally s...,https://www.wsj.com/politics/policy/white-hous...
46,63,2025-12-05-12-33-52 +0000,latimes,Netflix agrees to buy Warner Bros. in a $72-bi...,https://www.latimes.com/entertainment-arts/bus...
67,56,2025-12-05-11-42-00 +0000,wsj,The Supreme Court ruled that Texas could legal...,https://www.wsj.com/politics/policy/a-huge-red...
303,53,2025-12-04-19-19-46 +0000,nypost,Accused DC pipe-bomber Brian Cole Jr.’s nabbed...,https://nypost.com/2025/12/04/us-news/accused-...
367,48,2025-12-04-15-04-19 +0000,nypost,Putin’s alleged secret daughter tells Ukrainia...,https://nypost.com/2025/12/04/world-news/putin...
192,42,2025-12-05-00-22-00 +0000,wsj,"In a briefing for lawmakers, Navy Adm. Frank “...",https://www.wsj.com/politics/national-security...
347,41,2025-12-04-16-32-33 +0000,nyt,"Yasser Abu Shabab, Militant Leader Backed by I...",https://www.nytimes.com/2025/12/04/world/middl...
256,41,2025-12-04-21-58-39 +0000,nypost,CBS scrambles to sign Tony Dokoupil as anchor ...,https://nypost.com/2025/12/04/media/cbs-scramb...
193,36,2025-12-05-00-21-26 +0000,nypost,Trump sees peace ‘eventually’ in Ukraine after...,https://nypost.com/2025/12/04/world-news/trump...
178,36,2025-12-05-01-10-43 +0000,startribune,Twin Cities districts vow to block ICE access ...,https://www.startribune.com/twin-cities-school...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
